<a href="https://colab.research.google.com/github/ahsanGoheer/Autoencoder-Collection/blob/main/MNIST_Autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install Necessary Modules.

!pip install torch
!pip install torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Import Modules

import torch
from torchvision import datasets
from torchvision import transforms
import matplotlib.pyplot as plt

In [3]:
# Load the Mnist Dataset
 

# Prepare a tensor transformation for the images.
image_tensor_trfm = transforms.ToTensor()

# Download the MNIST Dataset.
mnist_dataset = datasets.MNIST(root = "./data",
                               train = True,
                               transform = image_tensor_trfm,
                               download = True)
# Create a Data Loader.
data_loader = torch.utils.data.DataLoader(dataset = mnist_dataset,
                                          batch_size = 32,
                                          shuffle = True)

100%|██████████| 9912422/9912422 [00:00<00:00, 282225119.43it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 120893906.01it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 152921866.15it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw




100%|██████████| 4542/4542 [00:00<00:00, 23204054.53it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [15]:
class AutoEncoder(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.encoder = torch.nn.Sequential(torch.nn.Linear(28 * 28, 128),
                                       torch.nn.ReLU(),
                                       torch.nn.Linear(128, 64),
                                       torch.nn.ReLU(),
                                       torch.nn.Linear(64, 32),
                                       torch.nn.ReLU(),
                                       torch.nn.Linear(32, 16),
                                       torch.nn.ReLU(),
                                       torch.nn.Linear(16, 8)
                                       )
    self.decoder = torch.nn.Sequential(torch.nn.Linear(8, 16),
                                       torch.nn.ReLU(),
                                       torch.nn.Linear(16, 32),
                                       torch.nn.ReLU(),
                                       torch.nn.Linear(32, 64),
                                       torch.nn.ReLU(),
                                       torch.nn.Linear(64, 128),
                                       torch.nn.ReLU(),
                                       torch.nn.Linear(128, 28 * 28)
                                       )
    
  def forward(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded




In [16]:
auto_encoder_model = AutoEncoder()
auto_encoder_model.cuda()

AutoEncoder(
  (encoder): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): ReLU()
    (6): Linear(in_features=32, out_features=16, bias=True)
    (7): ReLU()
    (8): Linear(in_features=16, out_features=8, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=8, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=64, bias=True)
    (5): ReLU()
    (6): Linear(in_features=64, out_features=128, bias=True)
    (7): ReLU()
    (8): Linear(in_features=128, out_features=784, bias=True)
  )
)

In [17]:
loss_func = torch.nn.MSELoss()
optimizer = torch.optim.Adam(auto_encoder_model.parameters(),
                             lr = 1e-1,
                             weight_decay = 1e-8
                             )

In [18]:
import numpy as np

In [ ]:
epochs = 20
outputs = []
losses = []
for epoch in range(epochs):
    print(f"Epoch:{epoch}")
    counter = 1
    for (image, _) in data_loader:
      
      # Reshaping the image to (-1, 784)
      image = image.reshape(-1, 28*28).cuda()
       
      # Output of Autoencoder
      reconstructed = auto_encoder_model(image)
       
      # Calculating the loss function
      loss = loss_func(reconstructed, image)
       
      # The gradients are set to zero,
      # the gradient is computed and stored.
      # .step() performs parameter update
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      print(f"Image:{counter} Loss:{loss}")
      counter+=1
      # Storing the losses in a list for plotting
      losses.append(loss)
    outputs.append((epochs, image, reconstructed))
 

Epoch:0
Image:1 Loss:0.07133113592863083
Image:2 Loss:0.06894353777170181
Image:3 Loss:0.06685762852430344
Image:4 Loss:0.07075859606266022
Image:5 Loss:0.07268468290567398
Image:6 Loss:0.07383985072374344
Image:7 Loss:0.06681365519762039
Image:8 Loss:0.06438234448432922
Image:9 Loss:0.06602165102958679
Image:10 Loss:0.06837021559476852
Image:11 Loss:0.06688612699508667
Image:12 Loss:0.06630778312683105
Image:13 Loss:0.07539310306310654
Image:14 Loss:0.07469932734966278
Image:15 Loss:0.0582379549741745
Image:16 Loss:0.06869851797819138
Image:17 Loss:0.06879521906375885
Image:18 Loss:0.067665696144104
Image:19 Loss:0.062214091420173645
Image:20 Loss:0.06684304028749466
Image:21 Loss:0.06425712257623672
Image:22 Loss:0.06791281700134277
Image:23 Loss:0.0711427628993988
Image:24 Loss:0.07141433656215668
Image:25 Loss:0.06851159781217575
Image:26 Loss:0.06526731699705124
Image:27 Loss:0.06485597044229507
Image:28 Loss:0.07098894566297531
Image:29 Loss:0.07233225554227829
Image:30 Loss:0.07

In [ ]:
# Defining the Plot Style
plt.style.use('fivethirtyeight')
plt.xlabel('Iterations')
plt.ylabel('Loss')
losses = [loss.cpu().data.numpy() for loss in losses]
# Plotting the last 100 values
plt.plot(losses[-100:])

In [ ]:
for i, item in enumerate(image):
   
  # Reshape the array for plotting
  item = item.reshape(-1, 28, 28)
  plt.imshow(item[0].cpu().data.numpy())
 
for i, item in enumerate(reconstructed):
  item = item.reshape(-1, 28, 28).cpu().data.numpy()
  plt.imshow(item[0])